# SQL. Анализ базы данных сервиса для чтения книг по подписке

## Описание данных
**Таблица `books`**. Содержит данные о книгах:
- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- num_pages — количество страниц
- publication_date — дата публикации книги
- publisher_id — идентификатор издателя

**Таблица `authors`**. Содержит данные об авторах:
- author_id — идентификатор автора
- author — имя автора

**Таблица `publishers`**. Содержит данные об издательствах:
- publisher_id — идентификатор издательства
- publisher — название издательства

**Таблица `ratings`**. Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

**Таблица `reviews`**. Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора
- book_id — идентификатор книги
- username — имя пользователя, написавшего обзор
- text — текст обзора

## Цели исследования
Проанализировать базу данных сервиса для чтения книг по подписке - информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db' # название базы данных
            } 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'], 
                                                         db_config['db']
                                                        )

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#### Исследование таблиц
Для каждой таблицы БД выведем первые 5 строк, посчитаем общее число записей и проверим записи на дубликаты

In [3]:
def run_sql(query):
    """
    Выполняет запрос к БД, выводит 10 строк результата
    
    query - текст запроса
    """
    data_raw = pd.io.sql.read_sql(query, con=engine)
    print(data_raw.head(10))        

In [4]:
def describe_table(table):
    """
    Для таблицы БД выводит общее число записей, число задублированных записей
    
    table - название таблицы
    """
    query = ''' SELECT * FROM ''' + table
    
    data_raw = pd.io.sql.read_sql(query, con=engine)
    print('Таблица', table, '\n')
    print(data_raw.head(), '\n')
    print('Число записей в таблице:', data_raw.shape, '\n')
    
    print('Дублированные записи:', '\n',
      data_raw[data_raw.duplicated()].count()
     )

In [5]:
describe_table('books')

Таблица books 

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268   

Число записей в таблице: (1000, 6) 

Дублированные записи: 
 book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64


In [6]:
describe_table('authors')

Таблица authors 

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd 

Число записей в таблице: (636, 2) 

Дублированные записи: 
 author_id    0
author       0
dtype: int64


In [7]:
describe_table('publishers')

Таблица publishers 

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company 

Число записей в таблице: (340, 2) 

Дублированные записи: 
 publisher_id    0
publisher       0
dtype: int64


In [8]:
describe_table('ratings')

Таблица ratings 

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2 

Число записей в таблице: (6456, 4) 

Дублированные записи: 
 rating_id    0
book_id      0
username     0
rating       0
dtype: int64


In [9]:
describe_table('reviews')

Таблица reviews 

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...   

Число записей в таблице: (2793, 4) 

Дублированные записи: 
 review_id    0
book_id      0
username     0
text         0
dtype: int64


#### Посчитайте, сколько книг вышло после 1 января 2000 года

In [10]:
query = ''' SELECT count(*)
            FROM books WHERE publication_date > '2000-01-01'
        '''
run_sql(query)

   count
0    819


Всего 819 книг вышло после 1 января 2000 г.

#### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [11]:
query = ''' SELECT b.book_id,
                    b.title,
                    r.review_count,
                    rr.rating_average
            FROM books b, 
                (SELECT book_id, count(review_id) review_count FROM reviews group by book_id) r,
                (SELECT book_id, avg(rating) rating_average FROM ratings group by book_id) rr
            where b.book_id = r.book_id
            and b.book_id = rr.book_id
            order by b.book_id
        '''
run_sql(query)

   book_id                                              title  review_count  \
0        1                                       'Salem's Lot             2   
1        2                 1 000 Places to See Before You Die             1   
2        3  13 Little Blue Envelopes (Little Blue Envelope...             3   
3        4  1491: New Revelations of the Americas Before C...             2   
4        5                                               1776             4   
5        6               1st to Die (Women's Murder Club  #1)             4   
6        7               2nd Chance (Women's Murder Club  #2)             3   
7        8              4th of July (Women's Murder Club  #4)             3   
8        9                                   A Beautiful Mind             2   
9       10                                 A Bend in the Road             3   

   rating_average  
0        3.666667  
1        2.500000  
2        4.666667  
3        4.500000  
4        4.000000  
5        3

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
query = ''' SELECT p.publisher_id, p.publisher, count(b.book_id) books_num
            FROM books b, publishers p
            WHERE b.publisher_id = p.publisher_id
            and b.num_pages > 50
            group by p.publisher_id
            order by books_num desc
        '''
run_sql(query)

   publisher_id                  publisher  books_num
0           212              Penguin Books         42
1           309                    Vintage         31
2           116   Grand Central Publishing         25
3           217           Penguin Classics         24
4            33           Ballantine Books         19
5            35                     Bantam         19
6            45                    Berkley         17
7            46              Berkley Books         14
8           284         St. Martin's Press         14
9           333  William Morrow Paperbacks         13


Издательство "Penguin Books" выпустило 42 книги толще 50 страниц

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
# вспомогательный запрос - средний рейтинг для книг с 50 и больше оценок
query = ''' SELECT b.book_id, b.title, avg(r.rating) avg_rating
                    FROM books b, ratings r
                    WHERE b.book_id = r.book_id
                    group by b.book_id, b.title
                    having count(rating_id) >= 50
            order by avg_rating desc
                    
        '''
pd.io.sql.read_sql(query, con=engine)

,book_id,title,avg_rating
0,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634
1,722,The Fellowship of the Ring (The Lord of the Ri...,4.391892
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500
3,656,The Book Thief,4.264151
4,300,Harry Potter and the Half-Blood Prince (Harry ...,4.246575
5,399,Little Women,4.192308
6,301,Harry Potter and the Order of the Phoenix (Har...,4.186667
7,750,The Hobbit or There and Back Again,4.125000
8,779,The Lightning Thief (Percy Jackson and the Oly...,4.080645
9,405,Lord of the Flies,3.901408


In [14]:
# к предыдущему запросу добавляем выбор автора и сортировку по среднему рейтингу
query = ''' SELECT a.author, rr.title, rr.avg_rating
            FROM authors a, 
            (SELECT b.book_id, b.title, avg(r.rating) avg_rating, b.author_id
                    FROM books b, ratings r
                    WHERE b.book_id = r.book_id
                    group by b.book_id, b.title
                    having count(rating_id) >= 50) rr
            WHERE a.author_id = rr.author_id
            order by rr.avg_rating desc
                    
        '''
pd.io.sql.read_sql(query, con=engine)

,author,title,avg_rating
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634
1,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,4.391892
2,J.K. Rowling/Mary GrandPré,Harry Potter and the Chamber of Secrets (Harry...,4.287500
3,Markus Zusak/Cao Xuân Việt Khương,The Book Thief,4.264151
4,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,4.246575
5,Louisa May Alcott,Little Women,4.192308
6,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...,4.186667
7,J.R.R. Tolkien,The Hobbit or There and Back Again,4.125000
8,Rick Riordan,The Lightning Thief (Percy Jackson and the Oly...,4.080645
9,William Golding,Lord of the Flies,3.901408


In [15]:
# посчитаем средний рейтинг по всем книгам автора
query = ''' SELECT a.author, avg(rr.avg_rating) average_rating
            FROM authors a, 
            (SELECT b.book_id, b.title, avg(r.rating) avg_rating, b.author_id
                    FROM books b, ratings r
                    WHERE b.book_id = r.book_id
                    group by b.book_id, b.title
                    having count(rating_id) >= 50) rr
            WHERE a.author_id = rr.author_id
            group by a.author_id
            order by average_rating desc                    
        '''
pd.io.sql.read_sql(query, con=engine)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [18]:
query = ''' SELECT avg(review_num) from (
            SELECT r.username, count(r.review_id) review_num
            FROM reviews r, 
                (SELECT username
                FROM ratings
                group by username
                having count(rating) > 50) rr
            where r.username = rr.username
            group by r.username
            ) t
        '''
run_sql(query)

         avg
0  24.333333


24.3 - среднее число обзоров от пользователей, которые поставили больше 50 оценок

## Выводы
Выполнено изучение таблиц БД, дубликатов записей не обнаружено.  
В результате анализа выявлено:
- всего 819 книг вышло после 1 января 2000 г.
- выполнен расчет числа обзоров и средней оценки для каждой книги
- издательство "Penguin Books" выпустило 42 книги толще 50 страниц
- J.K. Rowling/Mary GrandPré - максимальный средний рейтинг книг
- 24.3 - среднее число обзоров от пользователей, которые поставили больше 50 оценок